<table>
    <tr>
        <td style="text-align:left">
            <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9ItLTT_F-3Q30cu7ZCCoKmuFGBt22pe7pNA" alt="Logo Universidad" width="300"/>
        </td>
        <td>
            Departamento de Ciencias de la Computación y de la Decisión<br>
            Facultad de Minas<br>
            Universidad Nacional de Colombia<br>
            Optimizacion e IA 2024-2S<br><br>
            Docente: Maria Constanza Torres Madronero<br>
            <br>
            Contribuciones a la guia por: <br>
            - Deimer Miranda Montoya (2023)<br>
            - Luis Fernando Becerra Monsalve (2024)
        </td>    
        </td>    
    </tr>
</table>

### Optimización de funciones

Vamos a comparar el desempeño de los optimizadores gradiente descendente, algoritmo genético, enjambre de partículas y colonia de hormigas. Para ello usaremos una función no convexa con la cual trabajamos en la Practica 1.


In [ ]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
#Minimizacion de una funcion 2D
#f(x,y)=(x^2+y-11)^2+(x+y^2-7)^2
def objective_func(solution):
    term1 = (solution[0]**2 + solution[1] - 11)**2
    term2 = (solution[0] + solution[1]**2 - 7)**2
    return term1+term2

In [ ]:
#Grafica de la funcion
x = np.linspace(-5, 5, 1000)
y = np.linspace(-5, 5, 1000)
xv, yv = np.meshgrid(x, y)
feval = objective_func([xv,yv])
ax = plt.figure().add_subplot(111,projection='3d')
ax.plot_surface(xv, yv, feval, cmap=cm.jet)

### Gradiente descendiente

Recordemos el método del gradiente descendiente:
1. Necesitamos conocer la primera derivada de la función objetivo: el gradiente. Para nuestro ejemplo, dado que tenemos una función en dos dimensiones, el gradiente será un vector con dos componentes.


In [ ]:
#Funcion que evalua el gradiente en un punto solucion
def derivative(X):
  x = X[0]
  y = X[1]
  term0 = 4*x**3+4*x*y-42*x+2*y**2-14
  term1 = 2*x**2+4*x*y+4*y**3-26*y-22
  gradiente = np.array([term0,term1])
  return gradiente

2. El algoritmo iterativo: cada iteración proporciona una posible solución de la función de optimización. El punto inicial lo seleccionamos de forma aleatoria, y lo actualizamos en cada iteración restando el gradiente escalado con el paso de aprendizaje.

In [ ]:
#Funcion del gradiente descendente
def gradient_descent(objective_func, derivative, bounds, n_iter, step_size):
  # Generamos el punto inicial de forma aleatoria
  solution0 = bounds[:,0]+rand(len(bounds))*(bounds[:,1]-bounds[:,0])
  solution1 = bounds[:,0]+rand(len(bounds))*(bounds[:,1]-bounds[:,0])
  solution = np.array([solution0,solution1])
	# Algoritmo iterativo
  for i in range(n_iter):
		#1. Calculo del gradiente
    gradiente = derivative(solution)
		#2. Actualizacion de la solucion
    solution = solution - step_size * gradiente
    #3. Evaluacion de la solucion
    solution_eval = objective_func(solution)
    #print('>%d f(%s) = %.5f' % (i, solution, solution_eval))
  return solution

In [ ]:
# Deliminatos el rango de los datos
bounds = np.array([[-4.0, 4.0, ]])
# Definimos el numero de iteraciones
n_iter = 500
# Seleccionamos la longitud del paso
step_size = 0.01
# Aplicamos el algoritmo de gradiente descendente
solution = gradient_descent(objective_func, derivative, bounds, n_iter, step_size)
#Guardamos la solucion para la comparacion
xopGD, yopGD = solution
fevalGD = objective_func(solution)

In [ ]:
#Grafica de los contornos y punto optimo
plt.contourf(xv, yv, feval, levels=50, cmap='jet')
plt.scatter(xopGD, yopGD,c='white',marker='o')
plt.colorbar()
print(fevalGD)

### Algoritmo Genético
Existen diversas librerías con la implementación de los métodos metaheurísticos. Dado que es un área de desarrollo, es difícil encontrar librerías que integren una alta variedad de métodos y que permitan fácilmente su comparación.

En este practica usaremos primero la libreria PyGAD para aplicar el metodo basado en algoritmos geneticos para la minimizacion de nuestra funcion objetivo.

PyGAD implementa los algoritmo geneticos, puede ser combinado con algoritmos de aprendizaje de maquina, trabaja con Keras y Pytorch.

En el siguiente enlace pueden conocer un poco mas de esta libreria: [PyGAD](https://pygad.readthedocs.io/en/latest/index.html).

Estas librerías deben instalarse en el entorno de ejecución!!!

In [ ]:
#Instalacion de la libreria
!pip install pygad

In [ ]:
#Importar libreria pygad
import pygad

**Paso 1.** *Definir funcion fitness (funcion de aptitud).* Uno de los parámetros para entrenar el algoritmo genático es la función *fitness,* la cual debe ser definida por el usuario. Esta función debe ser una función de maximización, de tal forma que el valor más alto de aptitud sera retornado. La función puede retornar un unico valor o un arreglo. Esta función debe tener tres parámetros de entrada.
- *Parámetro 1.* La instancia de la clase GA
- *Parámetro 2.* La(s) solución(s) para calcular la función fitness
- *Parámetro 3.* Los índices de la solución en la población

In [ ]:
#
# Conversión del problema de minimización en uno de maximización

def fitness_func(ga_instance, solution, solution_idx):
    #Llamamos nuestra funcion objetivo
    output = objective_func(solution)
    #Calculamos el fitness (Convertimos el problema de minimizacion a maximizacion)
    fitness = 1.0/(output+1e4)
    return fitness

**Paso 2**. Preparamos los parámetros para correr el algoritmo genético.

In [ ]:
#
# Número de generaciones
num_generations = 500

# Número de soluciones a ser seleccionads como padres
num_parents_mating = 6

# Establecer la función fitness
fitness_function = fitness_func

# Número de soluciones por población
sol_per_pop  = 20

# Número de genes en la solución
# --> Por defecto, se establecen flotantes, pero se puede modificar int, uint
num_genes = 2

# Rangos iniciales para la población inicial
init_range_low = -4
init_range_high = 4

# Método para la seleccion de padres
# "rws": Ruleta
# "tournament": Torneo
parent_selection_type = 'tournament'

# Número de padres a mantener en la población
keep_parents = 3

# Tipo de cruce: en un solo punto, dos puntos, uniforme
crossover_type = 'single_point'

# Probabilidad para realizar el cruze
crossover_probability = 0.3

# Tipo de mutación
mutation_type = 'random'

# Probabilidad de mutación
mutation_probability = 0.05

In [ ]:
#Instanciar el Algoritmo
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       crossover_probability = crossover_probability,
                       mutation_type=mutation_type,
                       mutation_probability=mutation_probability)

**Paso 3.** Ejecución del algoritmo

In [ ]:
#Corremos el algoritmo
ga_instance.run()

**Paso 4.** Extracción de la mejor solución

In [ ]:
# Extraemos la mejor solución
solution, solution_fitness, solution_inx = ga_instance.best_solution()

##
print(f"Parameters of the best solution : {solution}")
print(f"Fitness value of the best solution = {solution_fitness}")
##
prediction = objective_func(solution)
print(f"Valor optimo para la funcion: = {prediction}")

# Guardamos la solucion para la comparacion
xopAG, yopAG = solution
fevalAG = objective_func(solution)


In [ ]:
# Graficar el avance
ga_instance.plot_fitness()

In [ ]:
#Grafica de los contornos y punto optimo
plt.contourf(xv, yv, feval, levels=50, cmap='jet')
plt.scatter(xopAG, yopAG,c='white',marker='o')
plt.colorbar()
print(fevalGD)